In [1]:
import mxnet as mx
import numpy as np
from sklearn.cluster import KMeans

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
prefix="cnn_models/resnet20"
epoch=124

sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)

batch_size=32
val_iter=mx.image.ImageIter(batch_size=batch_size,data_shape=(3,32,32),path_imgrec="dataset/cifar10_val.rec")
testarray=val_iter.next().data

test_iter=mx.io.NDArrayIter(testarray,batch_size=batch_size)
speedups=[]

In [3]:
def get_tensor(symbol_input):
    mod=mx.mod.Module(symbol=symbol_input, context=mx.cpu(),label_names=None)
    mod.bind(for_training=False, data_shapes=test_iter.provide_data)
    mod.set_params(args, auxs)
    return mod.predict(eval_data=test_iter)

In [ ]:
def get_channelwise_clustered(filters, shrink):
    shape = filters.shape
    print shape
    result=np.zeros(shape)
    n_clusters = int(shape[0] / shrink)
    print filters[:,0,:,:].shape
    print n_clusters

    for channel_idx in range(shape[1]):

        channelwise_filters = filters[:,channel_idx,:,:]
        cw_shape=channelwise_filters.shape

        channelwise_filters_shaped = channelwise_filters.reshape((cw_shape[0], cw_shape[1] * cw_shape[2]))
        estimator = KMeans(n_clusters=n_clusters)
        estimator.fit(channelwise_filters_shaped.asnumpy())

        cw_filter_kmean_indexes = estimator.predict(X=channelwise_filters_shaped.asnumpy())
        cw_filters_quantized = np.array([estimator.cluster_centers_[idx] for idx in cw_filter_kmean_indexes])

        result[:,channel_idx,:,:]=cw_filters_quantized.reshape(cw_shape)
    return mx.nd.array(result)
        

In [11]:
def get_quantized(filters, shrink=16):
    shape=filters.shape
    n_clusters=shape[0]*shape[1]/shrink

    filters_shaped=filters.reshape((shape[0] * shape[1], shape[2] * shape[3]))
    estimator = KMeans(n_clusters=n_clusters)
    estimator.fit(filters_shaped.asnumpy())

    filter_kmean_indexes = estimator.predict(X=filters_shaped.asnumpy())
    filters_quantized = np.array([estimator.cluster_centers_[idx] for idx in filter_kmean_indexes])
    filters_quantized = mx.nd.array(filters_quantized)



    fq= filters_quantized.reshape(shape)

    sum=0

    for channel in range(shape[1]):
        filters_in_channel = fq[:,channel,:,:]
        nclusters_channel = np.unique(filters_in_channel.asnumpy(),axis=0)
        sum+=nclusters_channel.shape[0]
        #print nclusters_channel.shape[0]


    speedups.append(float(shape[0]*shape[1])/sum)

    return fq

In [12]:
def get_layer_sqr_error(in_layer,out_layer, layer_weights, shrink):
    sym_in = sym.get_internals()[in_layer]
    sym_out_original = sym.get_internals()[out_layer]

    tensor_in = get_tensor(sym_in)
    tensor_out_original = get_tensor(sym_out_original)
    num_filter=tensor_out_original.shape[1]
    stride=tensor_in.shape[3]//tensor_out_original.shape[3]


    filters = args[layer_weights]

    filters_quantized_reshaped=get_quantized(filters,shrink)

    clustered_result = mx.ndarray.Convolution(data=tensor_in, weight=filters_quantized_reshaped, num_filter=num_filter,
                                              kernel=(3, 3), stride=(stride, stride), pad=(1, 1),no_bias=True, name="conv0")

    return np.square(tensor_out_original.asnumpy() - clustered_result.asnumpy()).mean()

In [15]:
get_layer_sqr_error('stage3_unit1_relu1_output','stage3_unit1_conv1_output','stage3_unit1_conv1_weight',2)

0.73077667

In [16]:
print speedups

[1.030699547055863]
